In [ ]:
"""
Executive Smart Calendar App (Highly Commented Version)

WHAT THIS APP DOES (in simple terms):

- Shows you a calendar GUI (window with a monthly calendar).
- Lets you click on any date and:
    - Add tasks
    - Edit tasks
    - Delete tasks
    - View details of tasks

- Each task can have:
    - Title
    - Time (start + end)
    - Priority (Low / Medium / High / Critical)
    - Status (Planned / In Progress / Done / Blocked)
    - Owner (person responsible)
    - Notes (extra details)

- It saves everything in two places:
    1. A local JSON file: tasks.json  (used by the app internally)
    2. An Excel file: executive_tasks.xlsx (for reporting / business use)

- Every time you add, edit, or delete a task:
    - tasks.json is updated
    - executive_tasks.xlsx is re-written with the updated list of tasks

IMPORTANT:
- If you manually delete the Excel file (executive_tasks.xlsx),
  it CANNOT be recovered by the app.
  The next time you change tasks, the app will create a fresh Excel file
  from the current task data.
"""

import tkinter as tk
from tkinter import ttk, messagebox  # ttk = nicer themed widgets
import calendar
import datetime
import json
import os

from openpyxl import Workbook  # Library to write Excel (.xlsx) files


# ---------------------------------------------------------------------
# FILE NAMES WE USE
# ---------------------------------------------------------------------

TASKS_FILE = "tasks.json"             # Where tasks are stored for the app
EXCEL_FILE = "executive_tasks.xlsx"   # Excel workbook with all duties


# ---------------------------------------------------------------------
# HELPER FUNCTIONS: LOADING AND SAVING TASKS (JSON)
# ---------------------------------------------------------------------

def load_tasks():
    """
    Load tasks from the JSON file into a Python dictionary.

    We store tasks in this structure:
        {
            "YYYY-MM-DD": [
                {
                    "title": "...",
                    "start_time": "...",
                    "end_time": "...",
                    "priority": "...",
                    "status": "...",
                    "owner": "...",
                    "notes": "..."
                },
                ...
            ],
            ...
        }

    If the file doesn't exist yet, we simply return an empty dictionary.
    """
    if not os.path.exists(TASKS_FILE):
        # No file yet = no tasks saved
        return {}

    try:
        with open(TASKS_FILE, "r", encoding="utf-8") as f:
            return json.load(f)
    except Exception as e:
        print("Error loading tasks.json:", e)
        # If something is wrong with the file, we just start fresh
        return {}


def save_tasks(tasks):
    """
    Save the tasks dictionary back into the JSON file.

    This overwrites the file with the latest version of all tasks.
    """
    try:
        with open(TASKS_FILE, "w", encoding="utf-8") as f:
            json.dump(tasks, f, indent=2)
    except Exception as e:
        print("Error saving tasks.json:", e)


# ---------------------------------------------------------------------
# POPUP WINDOW CLASS: ADDING / EDITING A SINGLE TASK
# ---------------------------------------------------------------------

class TaskDialog(tk.Toplevel):
    """
    This class creates a small popup window for adding or editing a task.

    - It shows fields for:
        - Title
        - Start Time
        - End Time
        - Priority
        - Status
        - Owner
        - Notes

    - When the user clicks "Save", we collect all the values and store them
      in self.result as a dictionary.

    - If the user clicks "Cancel" or closes the window, self.result stays None.
    """

    def __init__(self, parent, title="Task", task=None):
        """
        parent: The main app window (SmartCalendarApp).
        title:  Window title ("Add Task" or "Edit Task").
        task:   Existing task dict if editing; None if creating a new one.
        """
        super().__init__(parent)
        self.title(title)
        self.parent = parent
        self.result = None  # Will hold the final task dictionary

        # Set size of the popup window
        self.geometry("380x320")
        self.resizable(False, False)  # Window is not resizable

        # Make this window "modal":
        # user should handle this popup before going back to main window
        self.transient(parent)
        self.grab_set()

        # ------------------------------
        # Title field
        # ------------------------------
        tk.Label(self, text="Title:").pack(anchor="w", padx=10, pady=(10, 0))
        self.title_var = tk.StringVar(value=(task.get("title") if task else ""))
        tk.Entry(self, textvariable=self.title_var).pack(fill="x", padx=10)

        # ------------------------------
        # Time fields: Start and End
        # ------------------------------
        time_frame = tk.Frame(self)
        time_frame.pack(fill="x", padx=10, pady=5)

        tk.Label(time_frame, text="Start (HH:MM):").grid(row=0, column=0, sticky="w")
        self.start_var = tk.StringVar(value=(task.get("start_time") if task else ""))
        tk.Entry(time_frame, textvariable=self.start_var, width=8).grid(row=0, column=1, padx=(5, 15))

        tk.Label(time_frame, text="End (HH:MM):").grid(row=0, column=2, sticky="w")
        self.end_var = tk.StringVar(value=(task.get("end_time") if task else ""))
        tk.Entry(time_frame, textvariable=self.end_var, width=8).grid(row=0, column=3, padx=5)

        # ------------------------------
        # Priority and Status dropdowns
        # ------------------------------
        meta_frame = tk.Frame(self)
        meta_frame.pack(fill="x", padx=10, pady=5)

        # Priority combobox
        tk.Label(meta_frame, text="Priority:").grid(row=0, column=0, sticky="w")
        self.priority_var = tk.StringVar(value=(task.get("priority") if task else "Medium"))
        priority_box = ttk.Combobox(
            meta_frame,
            textvariable=self.priority_var,
            values=["Low", "Medium", "High", "Critical"],
            state="readonly",   # user must pick from the list
            width=10,
        )
        priority_box.grid(row=0, column=1, padx=(5, 15))

        # Status combobox
        tk.Label(meta_frame, text="Status:").grid(row=0, column=2, sticky="w")
        self.status_var = tk.StringVar(value=(task.get("status") if task else "Planned"))
        status_box = ttk.Combobox(
            meta_frame,
            textvariable=self.status_var,
            values=["Planned", "In Progress", "Done", "Blocked"],
            state="readonly",
            width=12,
        )
        status_box.grid(row=0, column=3, padx=5)

        # ------------------------------
        # Owner field (person responsible)
        # ------------------------------
        tk.Label(self, text="Owner (optional):").pack(anchor="w", padx=10)
        self.owner_var = tk.StringVar(value=(task.get("owner") if task else ""))
        tk.Entry(self, textvariable=self.owner_var).pack(fill="x", padx=10, pady=(0, 5))

        # ------------------------------
        # Notes (multi-line text box)
        # ------------------------------
        tk.Label(self, text="Notes:").pack(anchor="w", padx=10)
        self.notes_text = tk.Text(self, height=5)
        self.notes_text.pack(fill="both", padx=10, pady=(0, 10))
        if task and task.get("notes"):
            self.notes_text.insert("1.0", task["notes"])  # Insert existing notes

        # ------------------------------
        # Save / Cancel buttons
        # ------------------------------
        btn_frame = tk.Frame(self)
        btn_frame.pack(pady=(0, 10))

        tk.Button(btn_frame, text="Cancel", command=self.on_cancel).pack(side="right", padx=5)
        tk.Button(btn_frame, text="Save", command=self.on_ok).pack(side="right", padx=5)

        # Allow pressing Enter to save, Escape to cancel
        self.bind("<Return>", lambda e: self.on_ok())
        self.bind("<Escape>", lambda e: self.on_cancel())

        # Make sure window is visible and focused
        self.wait_visibility()
        self.focus_set()

    def validate_time(self, value: str) -> bool:
        """
        Check if the time is empty or in HH:MM format (24-hour clock).
        Example valid values: "09:30", "14:00", "" (empty allowed).
        """
        value = value.strip()
        if not value:
            return True  # empty is allowed
        try:
            datetime.datetime.strptime(value, "%H:%M")
            return True
        except ValueError:
            return False

    def on_ok(self):
        """
        Called when the user clicks "Save" or presses Enter.
        Validates the input and builds the result dictionary.
        """
        title = self.title_var.get().strip()
        if not title:
            messagebox.showwarning("Missing title", "Please enter a task title.")
            return  # Don't close the window yet

        start = self.start_var.get().strip()
        end = self.end_var.get().strip()

        # Validate times if provided
        if not self.validate_time(start):
            messagebox.showwarning("Invalid time", "Start time must be HH:MM (24-hour) or empty.")
            return

        if not self.validate_time(end):
            messagebox.showwarning("Invalid time", "End time must be HH:MM (24-hour) or empty.")
            return

        notes = self.notes_text.get("1.0", "end").strip()

        # Build the task dictionary that the main app will store
        self.result = {
            "title": title,
            "start_time": start,
            "end_time": end,
            "priority": self.priority_var.get(),
            "status": self.status_var.get(),
            "owner": self.owner_var.get().strip(),
            "notes": notes,
        }
        self.destroy()  # Close the popup

    def on_cancel(self):
        """
        Called when the user clicks "Cancel" or presses Escape.
        Just closes the window without saving anything.
        """
        self.result = None
        self.destroy()


# ---------------------------------------------------------------------
# MAIN APPLICATION CLASS: EXECUTIVE SMART CALENDAR
# ---------------------------------------------------------------------

class SmartCalendarApp(tk.Tk):
    """
    This is the main GUI application.

    It shows:
    - A monthly calendar (left side)
    - Tasks for the selected date (right side)
    - Filters for status, priority, search
    - KPIs (Today, Next 7 days, Overdue)
    - Excel info button
    """

    def __init__(self):
        super().__init__()

        self.title("Executive Smart Calendar")
        self.geometry("1000x640")  # Window size (width x height)

        # Load tasks from JSON file into memory (dictionary)
        self.tasks = load_tasks()

        # Keep track of currently displayed month and selected date
        today = datetime.date.today()
        self.current_year = today.year
        self.current_month = today.month
        self.selected_date = today

        # Configure layout grid of the main window
        # Two columns: left (calendar), right (task panel)
        self.columnconfigure(0, weight=3)
        self.columnconfigure(1, weight=2)
        self.rowconfigure(1, weight=1)  # main body row grows

        # Build all the UI parts
        self.create_header()
        self.create_calendar_view()
        self.create_task_view()
        self.create_kpi_view()

        # Draw initial calendar and tasks
        self.draw_calendar()
        self.update_task_list()
        self.update_kpis()

        # Also initialize Excel file so it's in sync
        self.export_tasks_to_excel()

    # ---------- Utility for dates ----------

    def date_key(self, date_obj: datetime.date) -> str:
        """
        Convert a date object into our standard string format: YYYY-MM-DD.
        This is used as the key in our tasks dictionary.
        """
        return date_obj.strftime("%Y-%m-%d")

    # ---------- Header (Month Navigation + Excel Info) ----------

    def create_header(self):
        """
        Creates the top header row:
        - Month navigation buttons (<, >, Today)
        - Month label (e.g., "November 2025")
        - Button that shows info about the Excel sheet
        """
        header = tk.Frame(self)
        header.grid(row=0, column=0, columnspan=2, sticky="ew", pady=5)
        header.columnconfigure(1, weight=1)

        tk.Button(header, text="<", width=3, command=lambda: self.change_month(-1)).grid(row=0, column=0, padx=5)

        self.month_label = tk.Label(header, text="", font=("Arial", 14, "bold"))
        self.month_label.grid(row=0, column=1)

        tk.Button(header, text=">", width=3, command=lambda: self.change_month(1)).grid(row=0, column=2, padx=5)
        tk.Button(header, text="Today", command=self.go_to_today).grid(row=0, column=3, padx=5)

        tk.Button(header, text="Excel Sheet Info", command=self.open_excel_info).grid(row=0, column=4, padx=10)

    # ---------- Calendar View (Left Panel) ----------

    def create_calendar_view(self):
        """
        Creates the frame that holds the month view calendar grid.
        """
        self.calendar_frame = tk.Frame(self, borderwidth=1, relief="solid")
        self.calendar_frame.grid(row=1, column=0, sticky="nsew", padx=10, pady=10)

        # Set up flexible row/column layout inside the calendar frame
        self.calendar_frame.rowconfigure(0, weight=0)  # day names row
        for r in range(1, 8):  # weeks
            self.calendar_frame.rowconfigure(r, weight=1)
        for c in range(7):      # days of week
            self.calendar_frame.columnconfigure(c, weight=1)

    # ---------- Task View (Right Panel) ----------

    def create_task_view(self):
        """
        Creates the panel on the right that shows tasks for the selected date,
        along with filters and buttons to manage tasks.
        """
        task_frame = tk.Frame(self, borderwidth=1, relief="solid")
        task_frame.grid(row=1, column=1, sticky="nsew", padx=(0, 10), pady=10)

        # Label that shows currently selected date (e.g., "Tasks for Monday, ...")
        self.selected_date_label = tk.Label(task_frame, text="", font=("Arial", 12, "bold"))
        self.selected_date_label.pack(anchor="w", padx=10, pady=(10, 5))

        # ------------------------------
        # Filters: Status, Priority, Search
        # ------------------------------
        filter_frame = tk.Frame(task_frame)
        filter_frame.pack(fill="x", padx=10, pady=(0, 5))

        tk.Label(filter_frame, text="Filter:").grid(row=0, column=0, sticky="w")

        # Dropdown to filter by task status
        self.filter_status_var = tk.StringVar(value="All")
        status_box = ttk.Combobox(
            filter_frame,
            textvariable=self.filter_status_var,
            values=["All", "Planned", "In Progress", "Done", "Blocked"],
            width=12,
            state="readonly",
        )
        status_box.grid(row=0, column=1, padx=5)
        status_box.bind("<<ComboboxSelected>>", lambda e: self.update_task_list())

        # Dropdown to filter by task priority
        self.filter_priority_var = tk.StringVar(value="All")
        prio_box = ttk.Combobox(
            filter_frame,
            textvariable=self.filter_priority_var,
            values=["All", "Low", "Medium", "High", "Critical"],
            width=10,
            state="readonly",
        )
        prio_box.grid(row=0, column=2, padx=5)
        prio_box.bind("<<ComboboxSelected>>", lambda e: self.update_task_list())

        # Text entry for keyword search (searches title, notes, owner)
        tk.Label(filter_frame, text="Search:").grid(row=0, column=3, sticky="e")
        self.search_var = tk.StringVar()
        search_entry = tk.Entry(filter_frame, textvariable=self.search_var, width=15)
        search_entry.grid(row=0, column=4, padx=5)
        search_entry.bind("<KeyRelease>", lambda e: self.update_task_list())

        # ------------------------------
        # Task list area (scrollable listbox)
        # ------------------------------
        list_frame = tk.Frame(task_frame)
        list_frame.pack(fill="both", expand=True, padx=10, pady=5)

        self.task_listbox = tk.Listbox(list_frame, height=15)
        self.task_listbox.pack(side="left", fill="both", expand=True)

        scrollbar = tk.Scrollbar(list_frame, orient="vertical", command=self.task_listbox.yview)
        scrollbar.pack(side="right", fill="y")
        self.task_listbox.config(yscrollcommand=scrollbar.set)

        # ------------------------------
        # Buttons under the list: Add/Edit/Delete/View
        # ------------------------------
        btn_frame = tk.Frame(task_frame)
        btn_frame.pack(pady=10)

        tk.Button(btn_frame, text="Add Task", width=12, command=self.add_task).grid(row=0, column=0, padx=5)
        tk.Button(btn_frame, text="Edit Task", width=12, command=self.edit_task).grid(row=0, column=1, padx=5)
        tk.Button(btn_frame, text="Delete Task", width=12, command=self.delete_task).grid(row=0, column=2, padx=5)
        tk.Button(btn_frame, text="View Details", width=12, command=self.view_task).grid(row=0, column=3, padx=5)

    # ---------- KPI View (Bottom Row) ----------

    def create_kpi_view(self):
        """
        Creates a small panel at the bottom showing summary metrics:
        - How many tasks today?
        - How many tasks in the next 7 days?
        - How many overdue tasks?
        """
        kpi_frame = tk.Frame(self, borderwidth=0)
        kpi_frame.grid(row=2, column=0, columnspan=2, sticky="ew", padx=10, pady=(0, 10))
        kpi_frame.columnconfigure(0, weight=1)
        kpi_frame.columnconfigure(1, weight=1)
        kpi_frame.columnconfigure(2, weight=1)

        self.kpi_today = tk.Label(kpi_frame, text="Today: 0 tasks", font=("Arial", 10, "bold"))
        self.kpi_today.grid(row=0, column=0, sticky="w")

        self.kpi_week = tk.Label(kpi_frame, text="Next 7 days: 0 tasks", font=("Arial", 10, "bold"))
        self.kpi_week.grid(row=0, column=1, sticky="w")

        self.kpi_overdue = tk.Label(kpi_frame, text="Overdue: 0 tasks", font=("Arial", 10, "bold"), fg="red")
        self.kpi_overdue.grid(row=0, column=2, sticky="w")

    # -----------------------------------------------------------------
    # CALENDAR DRAWING AND NAVIGATION
    # -----------------------------------------------------------------

    def draw_calendar(self):
        """
        Draws the calendar grid for the current month.
        Coloring rules:
        - Today's date: greenish background
        - Selected date: bluish background
        - Any date with tasks: yellowish background
        """
        # Clear any existing widgets inside the calendar frame
        for widget in self.calendar_frame.winfo_children():
            widget.destroy()

        # Set month label (e.g., "November 2025")
        month_name = calendar.month_name[self.current_month]
        self.month_label.config(text=f"{month_name} {self.current_year}")

        # Row 0: Day names
        days = ["Mon", "Tue", "Wed", "Thu", "Fri", "Sat", "Sun"]
        for col, name in enumerate(days):
            tk.Label(self.calendar_frame, text=name, font=("Arial", 10, "bold")).grid(row=0, column=col, pady=5)

        # calendar.Calendar gives us weeks with day numbers (0 = no day)
        cal = calendar.Calendar(firstweekday=0)  # Monday is the first day
        month_days = cal.monthdayscalendar(self.current_year, self.current_month)

        # For each "week" and each "day" create labels or buttons
        for row, week in enumerate(month_days, start=1):
            for col, day in enumerate(week):
                if day == 0:
                    # Empty cell (belongs to previous/next month)
                    tk.Label(self.calendar_frame, text="").grid(row=row, column=col, sticky="nsew", padx=2, pady=2)
                else:
                    date_obj = datetime.date(self.current_year, self.current_month, day)
                    key = self.date_key(date_obj)
                    has_tasks = key in self.tasks

                    is_today = (date_obj == datetime.date.today())
                    is_selected = (date_obj == self.selected_date)

                    # Choose background color based on date type
                    bg = "white"
                    if is_today:
                        bg = "#d1e7dd"      # light green for today
                    if is_selected:
                        bg = "#cfe2ff"      # light blue for selected date
                    if has_tasks and not is_selected:
                        bg = "#fff3cd"      # light yellow if tasks exist

                    # Use a button so clicking it selects the date
                    btn = tk.Button(
                        self.calendar_frame,
                        text=str(day),
                        bg=bg,
                        relief="ridge",
                        command=lambda d=date_obj: self.on_day_click(d),
                    )
                    btn.grid(row=row, column=col, sticky="nsew", padx=2, pady=2)

    def change_month(self, delta):
        """
        Move the calendar view forward or backward by 'delta' months.
        delta = +1 means next month, -1 means previous month.
        """
        month = self.current_month + delta
        year = self.current_year
        if month < 1:
            month = 12
            year -= 1
        elif month > 12:
            month = 1
            year += 1
        self.current_month = month
        self.current_year = year
        self.draw_calendar()

    def go_to_today(self):
        """
        Jump the calendar view and selection back to today's date.
        """
        today = datetime.date.today()
        self.current_year = today.year
        self.current_month = today.month
        self.selected_date = today
        self.draw_calendar()
        self.update_task_list()
        self.update_kpis()

    def on_day_click(self, date_obj):
        """
        Called when the user clicks a day button on the calendar.
        """
        self.selected_date = date_obj
        self.draw_calendar()
        self.update_task_list()
        self.update_kpis()

    # -----------------------------------------------------------------
    # TASK OPERATIONS: DISPLAY / ADD / EDIT / DELETE / VIEW
    # -----------------------------------------------------------------

    def update_task_list(self):
        """
        Refresh the task list on the right-hand side based on:
        - selected date
        - filters (status, priority)
        - search text
        """
        self.task_listbox.delete(0, "end")  # Clear everything

        key = self.date_key(self.selected_date)
        tasks_for_day = self.tasks.get(key, [])

        # Read current filter values
        status_filter = self.filter_status_var.get()
        priority_filter = self.filter_priority_var.get()
        search_text = self.search_var.get().strip().lower()

        # Apply filters and search
        filtered = []
        for t in tasks_for_day:
            if status_filter != "All" and t.get("status") != status_filter:
                continue
            if priority_filter != "All" and t.get("priority") != priority_filter:
                continue
            if search_text:
                combined = (t.get("title", "") +
                            " " + t.get("notes", "") +
                            " " + t.get("owner", "")).lower()
                if search_text not in combined:
                    continue
            filtered.append(t)

        # Sort tasks by start_time (so morning tasks appear first)
        def sort_key(t):
            st = t.get("start_time") or ""
            return st

        filtered = sorted(filtered, key=sort_key)

        # Format each task into a nice string and add to listbox
        for t in filtered:
            start = t.get("start_time", "")
            end = t.get("end_time", "")
            title = t.get("title", "")
            prio = t.get("priority", "")
            status = t.get("status", "")
            owner = t.get("owner", "")

            tag = f"[{prio}/{status}] "
            if owner:
                tag += f"({owner}) "

            if start or end:
                display = f"{start or '??'}-{end or '??'} {tag}{title}"
            else:
                display = f"{tag}{title}"
            self.task_listbox.insert("end", display)

        # Update the label with the selected date text
        date_str = self.selected_date.strftime("%A, %B %d, %Y")
        self.selected_date_label.config(text=f"Tasks for {date_str}")

    def add_task(self):
        """
        Open the TaskDialog to create a new task for the selected date.
        If the user saves, we add it to our dictionary and refresh.
        """
        dlg = TaskDialog(self, title="Add Task")
        self.wait_window(dlg)  # Wait until the dialog is closed
        if dlg.result:
            key = self.date_key(self.selected_date)
            self.tasks.setdefault(key, []).append(dlg.result)
            save_tasks(self.tasks)
            self.update_task_list()
            self.update_kpis()
            self.export_tasks_to_excel()

    def get_selected_task(self):
        """
        Helper function to get the currently selected task from the listbox.

        Returns:
            (key, tasks_for_day, index)
            or (None, None, None) if nothing is selected.
        """
        key = self.date_key(self.selected_date)
        tasks_for_day = self.tasks.get(key, [])
        idxs = self.task_listbox.curselection()

        if not idxs:
            messagebox.showinfo("No selection", "Please select a task first.")
            return None, None, None

        index = idxs[0]

        if index >= len(tasks_for_day):
            return None, None, None

        return key, tasks_for_day, index

    def edit_task(self):
        """
        Open the TaskDialog to edit the currently selected task.
        """
        key, tasks_for_day, index = self.get_selected_task()
        if key is None:
            return  # No selection

        task = tasks_for_day[index]
        dlg = TaskDialog(self, title="Edit Task", task=task)
        self.wait_window(dlg)
        if dlg.result:
            tasks_for_day[index] = dlg.result
            self.tasks[key] = tasks_for_day
            save_tasks(self.tasks)
            self.update_task_list()
            self.update_kpis()
            self.export_tasks_to_excel()

    def delete_task(self):
        """
        Delete the currently selected task after confirming.
        """
        key, tasks_for_day, index = self.get_selected_task()
        if key is None:
            return

        task = tasks_for_day[index]
        if messagebox.askyesno("Delete task", f"Delete '{task.get('title', '')}'?"):
            del tasks_for_day[index]
            if tasks_for_day:
                self.tasks[key] = tasks_for_day
            else:
                # If no tasks remain for that date, remove the date key entirely
                self.tasks.pop(key, None)
            save_tasks(self.tasks)
            self.update_task_list()
            self.update_kpis()
            self.export_tasks_to_excel()

    def view_task(self):
        """
        Show a popup with full details of the selected task.
        This is read-only, just for viewing.
        """
        key, tasks_for_day, index = self.get_selected_task()
        if key is None:
            return

        task = tasks_for_day[index]
        title = task.get("title", "")
        start = task.get("start_time", "")
        end = task.get("end_time", "")
        notes = task.get("notes", "")
        status = task.get("status", "")
        priority = task.get("priority", "")
        owner = task.get("owner", "")

        msg = f"Title: {title}\n"
        if start or end:
            msg += f"Time: {start or '??'} - {end or '??'}\n"
        msg += f"Priority: {priority}\nStatus: {status}\n"
        if owner:
            msg += f"Owner: {owner}\n"
        msg += "\nNotes:\n" + (notes or "(none)")
        messagebox.showinfo("Task details", msg)

    # -----------------------------------------------------------------
    # KPI CALCULATIONS
    # -----------------------------------------------------------------

    def update_kpis(self):
        """
        Update the KPIs displayed at the bottom:
        - Tasks today
        - Tasks in the next 7 days
        - Number of overdue tasks (past dates with non-Done tasks)
        """
        today = datetime.date.today()
        today_key = self.date_key(today)
        today_tasks = self.tasks.get(today_key, [])
        self.kpi_today.config(text=f"Today: {len(today_tasks)} tasks")

        # Count tasks for the next 7 days (including today)
        cnt_week = 0
        for i in range(7):
            d = today + datetime.timedelta(days=i)
            cnt_week += len(self.tasks.get(self.date_key(d), []))
        self.kpi_week.config(text=f"Next 7 days: {cnt_week} tasks")

        # Overdue = any task with date < today and status != "Done"
        overdue = 0
        for key, tasks in self.tasks.items():
            d = datetime.date.fromisoformat(key)
            if d < today:
                for t in tasks:
                    if t.get("status") != "Done":
                        overdue += 1
        self.kpi_overdue.config(text=f"Overdue: {overdue} tasks")

    # -----------------------------------------------------------------
    # EXCEL INTEGRATION
    # -----------------------------------------------------------------

    def export_tasks_to_excel(self):
        """
        Export all tasks into a single Excel file: executive_tasks.xlsx.

        - Each row = one task.
        - Columns: Date, Title, Start, End, Priority, Status, Owner, Notes

        If you delete the Excel file manually, it cannot be recovered
        by this app. A new one will be created from the current tasks
        the next time this function runs (e.g., when a task changes).
        """
        rows = []

        # Flatten the nested dictionary into a list of rows
        for date_str, task_list in self.tasks.items():
            for t in task_list:
                rows.append(
                    [
                        date_str,
                        t.get("title", ""),
                        t.get("start_time", ""),
                        t.get("end_time", ""),
                        t.get("priority", ""),
                        t.get("status", ""),
                        t.get("owner", ""),
                        t.get("notes", ""),
                    ]
                )

        # Create a new Excel workbook in memory
        wb = Workbook()
        ws = wb.active
        ws.title = "Tasks"

        # Header row
        headers = ["Date", "Title", "Start", "End", "Priority", "Status", "Owner", "Notes"]
        ws.append(headers)

        # Data rows
        for r in rows:
            ws.append(r)

        # Save workbook to disk (overwrites existing file)
        wb.save(EXCEL_FILE)

    def open_excel_info(self):
        """
        Show a popup with information about the Excel file:
        - Exact path
        - Behavior if you delete it
        """
        message = (
            f"The executive task log is saved as:\n\n{os.path.abspath(EXCEL_FILE)}\n\n"
            "It is updated automatically every time you add, edit, or delete a task.\n\n"
            "⚠ If you manually delete this .xlsx file from your computer, "
            "it cannot be recovered by this app. A new empty file will be "
            "created the next time you add or change a task (based on the "
            "current tasks stored in tasks.json)."
        )
        messagebox.showinfo("Excel task sheet", message)


# ---------------------------------------------------------------------
# APPLICATION ENTRY POINT
# ---------------------------------------------------------------------

if __name__ == "__main__":
    """
    This code runs only if you execute this file directly, for example:

        python executive_smart_calendar.py

    It creates the app object and starts the Tkinter main loop,
    which keeps the window open and responds to user actions.
    """
    app = SmartCalendarApp()
    app.mainloop()
